In [2]:
import tensorflow as tf
import numpy as np
import os
import time
import datetime
import preprocessing
from model import CharCNN
import pandas as pd
import numpy as np

In [6]:
def myload():
    filelist=['1F60A','1F62D','1F64F','1F605','1F614','1F629','2764']
    filelist_processed=['./tweet_test/data_processed_'+str(filename)+'.csv' for filename in filelist]
    mydata=pd.DataFrame(columns=['phrase','label'])
    for idx,fname in enumerate(filelist_processed):
        temp=pd.read_csv(fname, header=None).rename(columns={ 0:'phrase'})
        temp['label']=idx
        mydata=mydata.append(temp,sort=False)

    label_onehot=pd.get_dummies(mydata['label'])

    label_onehot=pd.get_dummies(mydata['label'])
    examples = []
    labels = []
    alphabet="abcdefghijklmnopqrstuvwxyz0123456789-,;.!?:'\"/\\|_@#$%^&*~`+-=<>()[]{}\n"
    
    cnt=0
    for idx,values in enumerate(zip(mydata['phrase'].values, label_onehot.values)):
        text_end_extracted=preprocessing.extract_end(list(values[0].lower()))
        padded=preprocessing.pad_sentence(text_end_extracted)
        text_int8_repr = preprocessing.string_to_int8_conversion(padded,alphabet)

        examples.append(text_int8_repr)
        labels.append(values[1])
        
        if idx%100000==0:
            print(idx,'completed')       
        
    return np.array(examples, dtype=np.int8), np.array(labels, dtype=np.int8)

In [15]:
# Parameters
# ==================================================

# Model Hyperparameters
dropout_keep_prob=0.5
l2_reg_lambda=0.0

# Training parameters
batch_size=128
num_epochs=1
evaluate_every=500
checkpoint_every=100

# Misc Parameters
allow_soft_placement=True
log_device_placement=False


# Data Preparation
# ==================================================
alphabet="abcdefghijklmnopqrstuvwxyz0123456789-,;.!?:'\"/\\|_@#$%^&*~`+-=<>()[]{}\n"

# Load data
print("Loading data...")
x, y = myload()

# Randomly shuffle data
np.random.seed(10)
shuffle_indices = np.random.permutation(np.arange(len(y)))
x_shuffled = x[shuffle_indices]
y_shuffled = y[shuffle_indices]
# Split train/test set
n_dev_samples = int(0.08*len(x))-1
# TODO: Create a fuckin' correct cross validation procedure
x_train, x_dev = x_shuffled[:-n_dev_samples], x_shuffled[-n_dev_samples:]
y_train, y_dev = y_shuffled[:-n_dev_samples], y_shuffled[-n_dev_samples:]
print("Train/Dev split: {:d}/{:d}".format(len(y_train), len(y_dev)))


# Training
# ==================================================

tf.reset_default_graph()
session_config=tf.ConfigProto( gpu_options=tf.GPUOptions(per_process_gpu_memory_fraction=0.5))
session_config.gpu_options.allow_growth = True
init = tf.global_variables_initializer()


with tf.Session(config=session_config) as sess:   
    sess.run(init)
    
    cnn = CharCNN(num_classes=7,sequence_max_length=1014)
    saver=tf.train.Saver()
    # Define Training procedure
    global_step = tf.Variable(0, name="global_step", trainable=False)
    optimizer = tf.train.AdamOptimizer(1e-4)
    grads_and_vars = optimizer.compute_gradients(cnn.loss)
    train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

    # Output directory for models and summaries
    timestamp = str(int(time.time()))
    out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
    print("Writing to {}\n".format(out_dir))

    # Initialize all variables
    sess.run(tf.initialize_all_variables())

    def train_step(x_batch, y_batch):
        """
        A single training step
        """
        feed_dict = {
          cnn.input_x: x_batch,
          cnn.input_y: y_batch,
          cnn.dropout_keep_prob: dropout_keep_prob
        }
        _, step, loss, accuracy = sess.run(
            [train_op, global_step, cnn.loss, cnn.accuracy],
            feed_dict)
        time_str = datetime.datetime.now().isoformat()
        print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))

    def dev_step(x_batch, y_batch):
        """
        Evaluates model on a dev set
        """
        dev_size = len(x_batch)
        max_batch_size = 500
        num_batches = int(dev_size/max_batch_size)
        acc = []
        losses = []
        print("Number of batches in dev set is " + str(num_batches))
        for i in range(num_batches):
            x_batch_dev, y_batch_dev = preprocessing.get_batched_one_hot(
                x_batch, y_batch, i * max_batch_size, (i + 1) * max_batch_size)
            feed_dict = {
              cnn.input_x: x_batch_dev,
              cnn.input_y: y_batch_dev,
              cnn.dropout_keep_prob: 1.0
            }
            step, loss, accuracy = sess.run(
                [global_step,  cnn.loss, cnn.accuracy],
                feed_dict)
            acc.append(accuracy)
            losses.append(loss)
            time_str = datetime.datetime.now().isoformat()
            print("batch " + str(i + 1) + " in dev >>" +
                  " {}: loss {:g}, acc {:g}".format(time_str, loss, accuracy))
        print("\nMean accuracy=" + str(sum(acc)/len(acc)))
        print("Mean loss=" + str(sum(losses)/len(losses)))


    # Generate batches in one-hot-encoding format
    batches = preprocessing.batch_iter(x_train, y_train, batch_size, num_epochs)
    # Training loop. For each batch...
    for batch in batches:
        x_batch, y_batch = zip(*batch)
        train_step(x_batch, y_batch)
        current_step = tf.train.global_step(sess, global_step)
        if current_step % evaluate_every == 0:
            print("\nEvaluation:")
            dev_step(x_dev, y_dev)
            print("")
            
    saver.save(sess, "./model/train.ckpt")

Loading data...
0 completed
100000 completed
200000 completed
300000 completed
Train/Dev split: 358158/31143
Writing to d:\dropbox\machine_learning\project\cnn_character_level_recognition\runs\1540218159

In epoch >> 1
num batches per epoch is: 2799
2018-10-22T23:22:41.501940: step 1, loss 4.89718, acc 0.0625
2018-10-22T23:22:42.107322: step 2, loss 2.73686, acc 0.195312
2018-10-22T23:22:42.874272: step 3, loss 2.54198, acc 0.234375
2018-10-22T23:22:43.375931: step 4, loss 2.52544, acc 0.234375
2018-10-22T23:22:43.902523: step 5, loss 2.43242, acc 0.28125
2018-10-22T23:22:44.373263: step 6, loss 2.50049, acc 0.164062
2018-10-22T23:22:44.880906: step 7, loss 2.24927, acc 0.203125
2018-10-22T23:22:45.382564: step 8, loss 2.27937, acc 0.164062
2018-10-22T23:22:46.004901: step 9, loss 2.10285, acc 0.21875
2018-10-22T23:22:46.541467: step 10, loss 2.11179, acc 0.234375
2018-10-22T23:22:47.038138: step 11, loss 2.14497, acc 0.15625
2018-10-22T23:22:47.567722: step 12, loss 2.01536, acc 0.234

2018-10-22T23:23:51.273808: step 127, loss 1.81718, acc 0.257812
2018-10-22T23:23:51.814363: step 128, loss 1.80625, acc 0.226562
2018-10-22T23:23:52.326994: step 129, loss 1.76516, acc 0.273438
2018-10-22T23:23:52.843612: step 130, loss 1.90768, acc 0.25
2018-10-22T23:23:53.416081: step 131, loss 1.8117, acc 0.273438
2018-10-22T23:23:53.986556: step 132, loss 1.81727, acc 0.226562
2018-10-22T23:23:54.495197: step 133, loss 1.7414, acc 0.296875
2018-10-22T23:23:54.958956: step 134, loss 1.83299, acc 0.257812
2018-10-22T23:23:55.430694: step 135, loss 1.82481, acc 0.289062
2018-10-22T23:23:55.997179: step 136, loss 1.74293, acc 0.328125
2018-10-22T23:23:56.702295: step 137, loss 1.75424, acc 0.304688
2018-10-22T23:23:57.261799: step 138, loss 1.75173, acc 0.304688
2018-10-22T23:23:57.787393: step 139, loss 1.85309, acc 0.296875
2018-10-22T23:23:58.313985: step 140, loss 1.87177, acc 0.273438
2018-10-22T23:23:58.827613: step 141, loss 1.85829, acc 0.28125
2018-10-22T23:23:59.419030: step

2018-10-22T23:25:04.199361: step 255, loss 1.81854, acc 0.296875
2018-10-22T23:25:04.740913: step 256, loss 1.77455, acc 0.265625
2018-10-22T23:25:05.395164: step 257, loss 1.72352, acc 0.34375
2018-10-22T23:25:06.009520: step 258, loss 1.78799, acc 0.21875
2018-10-22T23:25:06.568029: step 259, loss 1.77012, acc 0.351562
2018-10-22T23:25:07.352928: step 260, loss 1.84803, acc 0.257812
2018-10-22T23:25:08.119879: step 261, loss 1.84611, acc 0.21875
2018-10-22T23:25:08.709302: step 262, loss 1.79866, acc 0.195312
2018-10-22T23:25:09.350587: step 263, loss 1.80253, acc 0.273438
2018-10-22T23:25:10.085622: step 264, loss 1.77366, acc 0.25
2018-10-22T23:25:10.811681: step 265, loss 1.92035, acc 0.226562
2018-10-22T23:25:11.555692: step 266, loss 1.85889, acc 0.25
2018-10-22T23:25:12.265794: step 267, loss 1.78362, acc 0.257812
2018-10-22T23:25:12.964923: step 268, loss 1.80279, acc 0.289062
2018-10-22T23:25:13.638124: step 269, loss 1.80959, acc 0.25
2018-10-22T23:25:14.214093: step 270, lo

2018-10-22T23:26:16.732202: step 383, loss 1.65341, acc 0.382812
2018-10-22T23:26:17.249826: step 384, loss 1.76173, acc 0.335938
2018-10-22T23:26:17.704347: step 385, loss 1.79207, acc 0.296875
2018-10-22T23:26:18.148160: step 386, loss 1.80553, acc 0.257812
2018-10-22T23:26:18.859259: step 387, loss 1.76497, acc 0.328125
2018-10-22T23:26:19.466634: step 388, loss 1.73877, acc 0.304688
2018-10-22T23:26:20.270484: step 389, loss 1.81126, acc 0.234375
2018-10-22T23:26:21.151129: step 390, loss 1.82211, acc 0.273438
2018-10-22T23:26:21.845273: step 391, loss 1.79659, acc 0.28125
2018-10-22T23:26:22.468607: step 392, loss 1.73224, acc 0.351562
2018-10-22T23:26:23.206632: step 393, loss 1.83734, acc 0.242188
2018-10-22T23:26:23.823982: step 394, loss 1.70305, acc 0.328125
2018-10-22T23:26:24.372515: step 395, loss 1.90916, acc 0.242188
2018-10-22T23:26:24.892125: step 396, loss 1.73888, acc 0.320312
2018-10-22T23:26:25.408745: step 397, loss 1.80357, acc 0.289062
2018-10-22T23:26:25.953289

batch 9 in dev >> 2018-10-22T23:27:37.155729: loss 1.77652, acc 0.268
batch 10 in dev >> 2018-10-22T23:27:38.689626: loss 1.81338, acc 0.274
batch 11 in dev >> 2018-10-22T23:27:40.251450: loss 1.76451, acc 0.316
batch 12 in dev >> 2018-10-22T23:27:42.244123: loss 1.75273, acc 0.32
batch 13 in dev >> 2018-10-22T23:27:43.966516: loss 1.76606, acc 0.286
batch 14 in dev >> 2018-10-22T23:27:45.684922: loss 1.76849, acc 0.328
batch 15 in dev >> 2018-10-22T23:27:47.518020: loss 1.7629, acc 0.316
batch 16 in dev >> 2018-10-22T23:27:49.341147: loss 1.75598, acc 0.292
batch 17 in dev >> 2018-10-22T23:27:51.113408: loss 1.77303, acc 0.31
batch 18 in dev >> 2018-10-22T23:27:52.846773: loss 1.80173, acc 0.3
batch 19 in dev >> 2018-10-22T23:27:54.487386: loss 1.80619, acc 0.29
batch 20 in dev >> 2018-10-22T23:27:56.068159: loss 1.76893, acc 0.304
batch 21 in dev >> 2018-10-22T23:27:57.729717: loss 1.75142, acc 0.316
batch 22 in dev >> 2018-10-22T23:27:59.455104: loss 1.73336, acc 0.346
batch 23 in d

...

2018-10-22T23:49:05.324818: step 2256, loss 1.66433, acc 0.335938
2018-10-22T23:49:05.853407: step 2257, loss 1.68017, acc 0.414062
2018-10-22T23:49:06.424877: step 2258, loss 1.69559, acc 0.320312
2018-10-22T23:49:06.916561: step 2259, loss 1.66178, acc 0.34375
2018-10-22T23:49:07.560838: step 2260, loss 1.72258, acc 0.320312
2018-10-22T23:49:08.160237: step 2261, loss 1.74185, acc 0.328125
2018-10-22T23:49:08.927186: step 2262, loss 1.56296, acc 0.460938
2018-10-22T23:49:09.766941: step 2263, loss 1.66345, acc 0.359375
2018-10-22T23:49:10.442135: step 2264, loss 1.56921, acc 0.460938
2018-10-22T23:49:11.090403: step 2265, loss 1.66621, acc 0.382812
2018-10-22T23:49:11.642924: step 2266, loss 1.7663, acc 0.320312
2018-10-22T23:49:12.095714: step 2267, loss 1.65683, acc 0.359375
2018-10-22T23:49:12.545511: step 2268, loss 1.5173, acc 0.453125
2018-10-22T23:49:12.990322: step 2269, loss 1.70045, acc 0.328125
2018-10-22T23:49:13.477020: step 2270, loss 1.76204, acc 0.3125
2018-10-22T23:4

2018-10-22T23:50:07.975613: step 2382, loss 1.74838, acc 0.367188
2018-10-22T23:50:08.434386: step 2383, loss 1.6491, acc 0.382812
2018-10-22T23:50:08.909116: step 2384, loss 1.78334, acc 0.28125
2018-10-22T23:50:09.367890: step 2385, loss 1.66533, acc 0.367188
2018-10-22T23:50:09.861570: step 2386, loss 1.68438, acc 0.390625
2018-10-22T23:50:10.287430: step 2387, loss 1.82526, acc 0.296875
2018-10-22T23:50:10.813025: step 2388, loss 1.74245, acc 0.34375
2018-10-22T23:50:11.343607: step 2389, loss 1.70932, acc 0.359375
2018-10-22T23:50:11.793404: step 2390, loss 1.72285, acc 0.367188
2018-10-22T23:50:12.245196: step 2391, loss 1.67197, acc 0.34375
2018-10-22T23:50:12.695990: step 2392, loss 1.70911, acc 0.382812
2018-10-22T23:50:13.164737: step 2393, loss 1.60092, acc 0.367188
2018-10-22T23:50:13.594588: step 2394, loss 1.77611, acc 0.367188
2018-10-22T23:50:14.067324: step 2395, loss 1.63646, acc 0.429688
2018-10-22T23:50:14.493185: step 2396, loss 1.687, acc 0.375
2018-10-22T23:50:14

batch 6 in dev >> 2018-10-22T23:51:13.990043: loss 1.66072, acc 0.374
batch 7 in dev >> 2018-10-22T23:51:15.598742: loss 1.67911, acc 0.386
batch 8 in dev >> 2018-10-22T23:51:17.270272: loss 1.63498, acc 0.38
batch 9 in dev >> 2018-10-22T23:51:18.943798: loss 1.69941, acc 0.344
batch 10 in dev >> 2018-10-22T23:51:20.578428: loss 1.72684, acc 0.348
batch 11 in dev >> 2018-10-22T23:51:22.305808: loss 1.6473, acc 0.406
batch 12 in dev >> 2018-10-22T23:51:23.982326: loss 1.66498, acc 0.38
batch 13 in dev >> 2018-10-22T23:51:25.809440: loss 1.65192, acc 0.366
batch 14 in dev >> 2018-10-22T23:51:27.340347: loss 1.67367, acc 0.388
batch 15 in dev >> 2018-10-22T23:51:28.932091: loss 1.67486, acc 0.366
batch 16 in dev >> 2018-10-22T23:51:30.493915: loss 1.63729, acc 0.384
batch 17 in dev >> 2018-10-22T23:51:32.089648: loss 1.68856, acc 0.37
batch 18 in dev >> 2018-10-22T23:51:33.741232: loss 1.73123, acc 0.364
batch 19 in dev >> 2018-10-22T23:51:35.250198: loss 1.66599, acc 0.368
batch 20 in de

2018-10-22T23:53:20.895507: step 2564, loss 1.69499, acc 0.335938
2018-10-22T23:53:21.336326: step 2565, loss 1.70679, acc 0.359375
2018-10-22T23:53:21.804076: step 2566, loss 1.76417, acc 0.382812
2018-10-22T23:53:22.309724: step 2567, loss 1.57991, acc 0.4375
2018-10-22T23:53:22.817367: step 2568, loss 1.75173, acc 0.296875
2018-10-22T23:53:23.286113: step 2569, loss 1.75693, acc 0.328125
2018-10-22T23:53:23.815698: step 2570, loss 1.77571, acc 0.3125
2018-10-22T23:53:24.287437: step 2571, loss 1.75574, acc 0.367188
2018-10-22T23:53:24.786103: step 2572, loss 1.77153, acc 0.328125
2018-10-22T23:53:25.268812: step 2573, loss 1.67012, acc 0.375
2018-10-22T23:53:25.696669: step 2574, loss 1.71858, acc 0.3125
2018-10-22T23:53:26.177382: step 2575, loss 1.6281, acc 0.390625
2018-10-22T23:53:26.620199: step 2576, loss 1.69687, acc 0.34375
2018-10-22T23:53:27.101911: step 2577, loss 1.68433, acc 0.34375
2018-10-22T23:53:27.611548: step 2578, loss 1.75952, acc 0.328125
2018-10-22T23:53:28.02

2018-10-22T23:54:21.680908: step 2690, loss 1.68834, acc 0.359375
2018-10-22T23:54:22.099789: step 2691, loss 1.68472, acc 0.390625
2018-10-22T23:54:22.529639: step 2692, loss 1.75717, acc 0.304688
2018-10-22T23:54:23.014342: step 2693, loss 1.69142, acc 0.351562
2018-10-22T23:54:23.446188: step 2694, loss 1.69274, acc 0.367188
2018-10-22T23:54:23.946849: step 2695, loss 1.73501, acc 0.359375
2018-10-22T23:54:24.396647: step 2696, loss 1.56069, acc 0.445312
2018-10-22T23:54:24.817522: step 2697, loss 1.68229, acc 0.367188
2018-10-22T23:54:25.282279: step 2698, loss 1.71529, acc 0.375
2018-10-22T23:54:25.757009: step 2699, loss 1.6632, acc 0.382812
2018-10-22T23:54:26.290583: step 2700, loss 1.64254, acc 0.375
2018-10-22T23:54:26.763319: step 2701, loss 1.7772, acc 0.335938
2018-10-22T23:54:27.239047: step 2702, loss 1.71138, acc 0.351562
2018-10-22T23:54:27.686849: step 2703, loss 1.5887, acc 0.421875
2018-10-22T23:54:28.166568: step 2704, loss 1.74885, acc 0.34375
2018-10-22T23:54:28.

In [17]:
def single_transform(values):
    alphabet="abcdefghijklmnopqrstuvwxyz0123456789-,;.!?:'\"/\\|_@#$%^&*~`+-=<>()[]{}\n"
    text_end_extracted=preprocessing.extract_end(list(values.lower()))
    padded=preprocessing.pad_sentence(text_end_extracted)
    text_int8_repr = preprocessing.string_to_int8_conversion(padded,alphabet)
    feed_value=np.array(text_int8_repr, dtype=np.int8)
    
    alphabet = "abcdefghijklmnopqrstuvwxyz0123456789-,;.!?:'\"/\\|_@#$%^&*~`+-=<>()[]{}\n"
        
    x_batch_one_hot = np.zeros(shape=[1, len(alphabet), len(feed_value), 1])
    for char_pos_in_seq, char_seq_char_ind in enumerate(feed_value):
        if char_seq_char_ind != -1:
            x_batch_one_hot[0][char_seq_char_ind][char_pos_in_seq][0] = 1

    return x_batch_one_hot

In [24]:
def pred(input_text):
    tf.reset_default_graph()
    session_config=tf.ConfigProto( gpu_options=tf.GPUOptions(per_process_gpu_memory_fraction=0.5))
    session_config.gpu_options.allow_growth = True
    saver=tf.train.import_meta_graph("./model/train.ckpt.meta")

    with tf.Session(config=session_config) as sess:      
        saver.restore(sess,'./model/train.ckpt')
        input_x = tf.get_default_graph().get_tensor_by_name('input_x:0')
        dropout_keep_prob= tf.get_default_graph().get_tensor_by_name('dropout_keep_prob:0')
        answer=sess.run(['fc-3/predictions:0'] , feed_dict={ input_x : single_transform(input_text) , dropout_keep_prob: 1.0})
        
    sentiment_list={
    0: 'SMILING FACE WITH SMILING EYES',
    1: 'LOUDLY CRYING FACE',
    2: 'PERSON WITH FOLDED HANDS',
    3: 'SMILING FACE WITH OPEN MOUTH AND COLD SWEAT',
    4:'PENSIVE FACE',
    5: 'WEARY FACE',
    6: 'HEAVY BLACK HEART',
    }
        
    return sentiment_list[answer[0][0]]

In [38]:
pred('I am praying')

INFO:tensorflow:Restoring parameters from ./model/train.ckpt


'HEAVY BLACK HEART'